In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from pymongo import MongoClient
import pandas as pd
import json
import re
import requests

In [2]:
# Connect to mongodb
client = MongoClient('localhost', 27017)
# Create new database
db = client['Project']

In [15]:
op = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# Scrape the top selling games page

In [16]:
# get the top selling games page
driver.get("https://store.steampowered.com/search/?ignore_preferences=1&untags=493&category1=998&supportedlang=english&filter=globaltopsellers&ndl=1")

In [17]:
# Add cookies to be able to see also violent content
driver.add_cookie({'name' : 'birthtime', 'value' : '283993201'})
driver.add_cookie({'name' : 'mature_content', 'value': '1'})

In [7]:
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [18]:
soup = BeautifulSoup(driver.page_source, features="html.parser")

In [19]:
top_games = []
rank = 1
div = soup.find("div", id="search_resultsRows")
for a in div.findAll("a"):
    game = {}
    game['rank'] = rank
    rank += 1
    # append serial Number
    game['SN'] = a['data-ds-appid']
    # append Title
    game['title'] = a.find("span", class_="title").text
    # append release date
    game['relDate'] = a.find("div", class_="col search_released responsive_secondrow").text
    # append price
    price = a.find("div", class_="col search_price responsive_secondrow")
    if price != None:
        game['price'] = price.text.strip()
    else:
        try:
            game['price'] = a.find("div", class_="col search_price discounted responsive_secondrow").find("span").text
        except: game['price'] = None
    # append dictionary to list
    top_games.append(game)

In [20]:
len(top_games)

5771

# Scrape games descriptions and genres

In [11]:
for game in top_games:
    # Open the link thanks to the Serial Number
    driver.get("https://store.steampowered.com/app/" + game['SN'])
    soup = BeautifulSoup(driver.page_source, features="html.parser")
    # If it is not a game (eg: Steam Deck) append an attribute "toDrop" and continue
    if "game_bg" not in soup.find("body")["class"]:
        game['toDrop'] = True
        continue
    # find the percentage of RECENT POSITIVE steam reviews
    try:
        text = soup.findAll("span", class_="responsive_reviewdesc_short")[1].text
        # get only text inside parenthesis
        string = re.findall("\(([^)]+)\)", text)[0].split(" ")
        game['posPercRec'] = string[0]
        # we use sub to eliminate the thousand indicator
        game['totRec'] = re.sub(",","", string[2])
    except:
        game['posPercRec'] = None
        game['totRec'] = None
    # find the percentage of TOTAL POSITIVE steam reviews
    try:
        text = soup.findAll("span", class_="responsive_reviewdesc_short")[0].text
        string = re.findall("\(([^)]+)\)", text)[0].split(" ")
        game['posPercTot'] = string[0]
        game['totTot'] = re.sub(",","", string[2])
    except:
        game['posPercTot'] = None
        game['totTot'] = None
    # find the GENRES
    try:
        div = soup.find("div", string="Etichette popolari definite dagli utenti per questo prodotto:").find_next_siblings()[0]
        game['genres'] = []
        for a in div.findAll("a"):
            game['genres'].append(a.text.strip())
    except:
        game['genres'] = [None]

# Upload to Database

In [12]:
# Create new database collection
Steam = db.Steam
# upload to database
Steam.insert_many(top_games)